# FairWorkflows execution demo

## Define the steps of your workflow
Each step should be its own function. Mark the function as such with the @fairstep decorator.

In [1]:
from fairworkflows import is_fairworkflow, is_fairstep

In [2]:
@is_fairstep(label='Addition')
def add(a:float, b:float) -> float:
    """Adding up numbers!"""
    return a + b

In [3]:
@is_fairstep(label='Subtraction')
def sub(a: float, b: float) -> float:
    """Subtracting numbers."""
    return a - b

In [4]:
@is_fairstep(label='Multiplication')
def mul(a: float, b: float) -> float:
    """Multiplying numbers."""
    return a * b

In [5]:
@is_fairstep(label='A strange step with little use')
def weird(a: float, b:float) -> float:
    """A weird function"""
    return a * 2 + b * 4
    

## Define your workflow using @fairworkflow
Now write a function which describes your workflow. Mark this function with the @fairworkflow decorator.

In [6]:
@is_fairworkflow(label='My Workflow')
def my_workflow(in1, in2, in3):
    """
    A simple addition, subtraction, multiplication workflow
    """
    t1 = add(in1, in2)
    t2 = sub(in1, in2)
    t3 = mul(weird(t1, in3), t2)
    return t3

## Create an instance of your workflow and display it

In [7]:
fw = my_workflow(1, 4, 3)
type(fw)

fairworkflows.fairworkflow.FairWorkflow

In [8]:
fw.display()

| workflow |
| --- |
| ![workflow workflow](control-workflow.svg) |

## Publish the (prospective) workflow
You may publish the workflow, and its steps, as nanopublications in the usual manner:

In [9]:
#fw.publish_as_nanopub()

Be warned though - the above will keep publishing to the 'real' nanopub server network. For testing you may prefer to publish to the test servers as follows (note that this will refuse to publish a workflow you have already published :

In [10]:
fw.publish_as_nanopub(use_test_server=True)

Published to http://purl.org/np/RAiPTeYNAhFAfYft6htEHljFXxR8bCaRxH2Ais1b9prcA
Published concept to http://purl.org/np/RAiPTeYNAhFAfYft6htEHljFXxR8bCaRxH2Ais1b9prcA#step
Published to http://purl.org/np/RAGFccFuQRkxa5R5AjDUN6SwF3LQE8pAc3bIAdp9KD16I
Published concept to http://purl.org/np/RAGFccFuQRkxa5R5AjDUN6SwF3LQE8pAc3bIAdp9KD16I#step
Published to http://purl.org/np/RAoGKox_-enW_2DrM6qcny4wM6IYfYndsp8ABOaxLu754
Published concept to http://purl.org/np/RAoGKox_-enW_2DrM6qcny4wM6IYfYndsp8ABOaxLu754#step
Published to http://purl.org/np/RANnEexpRFhnlVx5Myv-ZVJx07dSFrhM_BP0RSqW39_Ag
Published concept to http://purl.org/np/RANnEexpRFhnlVx5Myv-ZVJx07dSFrhM_BP0RSqW39_Ag#step
Published to http://purl.org/np/RA2BTYCsFPkkdct55P2Am_l3fxHQZp1ariOPkQoKx_XaY
Published concept to http://purl.org/np/RA2BTYCsFPkkdct55P2Am_l3fxHQZp1ariOPkQoKx_XaY#plan


{'nanopub_uri': 'http://purl.org/np/RA2BTYCsFPkkdct55P2Am_l3fxHQZp1ariOPkQoKx_XaY',
 'concept_uri': 'http://purl.org/np/RA2BTYCsFPkkdct55P2Am_l3fxHQZp1ariOPkQoKx_XaY#plan'}

You can then find your nanopublications by replacing the base of the URI with http://test-server.nanopubs.lod.labs.vu.nl/

## Execute your workflow using .execute()
Set num_threads greater than 1 if you wish to exploit parallelisation in your workflow. The retrospective provenance is also returned as a (nano) Publication object, that can optionally be published.

In [13]:
result, prov = fw.execute(num_threads=2)
result

-66

 ## Retrospective prov
 The retrospective prov object is not yet implemented, pending one of the RDF tasks set out in the planning document. For now, you may see simply the provenance trace for this run. We wish to turn this into a (Plex) RDF nanopublication:

In [14]:
type(prov)

nanopub.publication.Publication

In [16]:
print(prov)

Original source URI = None
@prefix : <http://purl.org/nanopub/temp/mynanopub#> .
@prefix np: <http://www.nanopub.org/nschema#> .
@prefix prov: <http://www.w3.org/ns/prov#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

:Head {
    : a np:Nanopublication ;
        np:hasAssertion :assertion ;
        np:hasProvenance :provenance ;
        np:hasPublicationInfo :pubInfo .
}

:assertion {
    :retroprov a prov:Activity ;
        rdfs:label """2021-01-28 12:01:55,933 - job             1: mul(22, -3) -> -66
2021-01-28 12:01:55,933 - job             2: weird(5, 3) -> 22
2021-01-28 12:01:55,933 - job             3: add(1, 4) -> 5
2021-01-28 12:01:55,933 - job             4: sub(1, 4) -> -3
2021-01-28 12:01:55,936 - result          1 [mul(22, -3) -> -66]: done -> -66
2021-01-28 12:01:55,936 - -end-of-queue-
""" ;
        prov:wasDerivedFrom <http://purl.org/np/RA2BTYCsFPkkdct55P2Am_l3fxHQZp1ariOPkQoKx_XaY#plan> .
}

:pubInfo {
    